<a href="https://colab.research.google.com/github/Kimttaeshik/Pupil-Reflex-AI/blob/main/PupilReflexAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

부연 설명
os와 cv2 라이브러리를 사용해 데이터 디렉토리를 탐색하고 이미지를 로드합니다.
레이블 분류: 폴더 구조가 0/ (비정상)과 1/ (정상)처럼 구성되어 있다고 가정합니다.
이미지를 정규화(0~1)하여 학습 안정성을 높입니다.
이미지 크기 조정: img_size=(64, 64)는 기본 크기입니다. GPU 메모리와 모델 성능에 맞게 크기를 조정할 수 있습니다.

In [ ]:
import os
import cv2
import numpy as np

# 데이터 로드 및 전처리 함수
def load_data(data_dir, img_size=(64, 64)):
    """
    데이터셋 디렉토리에서 이미지를 로드하고 전처리합니다.
    - 이미지를 회색조로 변환
    - 지정된 크기로 리사이즈
    - 정규화 (0~1 사이 값으로 변환)

    Parameters:
        data_dir (str): 데이터셋 디렉토리 경로
        img_size (tuple): 이미지 크기 (기본값 64x64)

    Returns:
        images (np.array): 전처리된 이미지 배열
        labels (np.array): 이미지 레이블 배열
    """
    images = []
    labels = []
    for label, sub_dir in enumerate(os.listdir(data_dir)):  # 서브 디렉토리를 기준으로 레이블 설정
        sub_dir_path = os.path.join(data_dir, sub_dir)
        for img_file in os.listdir(sub_dir_path):
            img_path = os.path.join(sub_dir_path, img_file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # 회색조 변환
            img = cv2.resize(img, img_size)  # 리사이즈
            images.append(img)
            labels.append(label)
    images = np.array(images) / 255.0  # 정규화
    labels = np.array(labels)
    return images, labels

부연 설명
train_test_split를 사용해 데이터를 80% 학습, 20% 테스트로 나눕니다.
CNN 입력 형식: (64, 64, 1)은 CNN 모델이 요구하는 이미지 형식입니다. 마지막 차원 1은 회색조 이미지를 나타냅니다.
데이터 불균형이 있다면, stratify=labels를 추가해 균형을 맞출 수 있습니다.


In [ ]:
from sklearn.model_selection import train_test_split

# 데이터 로드
data_dir = 'path_to_dataset'  # 데이터셋 경로
images, labels = load_data(data_dir)

# 데이터 분리 (학습/테스트 세트)
X_train, X_test, y_train, y_test = train_test_split(
    images, labels, test_size=0.2, random_state=42
)

# CNN 입력 형식으로 데이터 변환
X_train = X_train.reshape(-1, 64, 64, 1)
X_test = X_test.reshape(-1, 64, 64, 1)


부연 설명
Conv2D와 MaxPooling2D: 특징 추출과 다운샘플링을 담당합니다.
Dropout: 과적합 방지를 위해 사용됩니다. Dropout 비율은 데이터 크기와 모델 복잡도에 따라 조정 가능합니다.
Data Augmentation: 데이터 양이 적을 경우, 데이터 증강을 통해 모델의 일반화를 개선합니다. 필요에 따라 증강 파라미터를 조정하세요.
Epochs: 학습 데이터 크기에 따라 10~50 epochs를 실험해 최적 값을 찾으세요.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# CNN 모델 생성
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
    MaxPooling2D((2, 2)),
    Dropout(0.25),  # 과적합 방지
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # 과적합 방지
    Dense(1, activation='sigmoid')  # 이진 분류
])

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 데이터 증강
datagen = ImageDataGenerator(
    rotation_range=10,  # 회전
    width_shift_range=0.1,  # 가로 이동
    height_shift_range=0.1,  # 세로 이동
    horizontal_flip=True  # 좌우 반전
)
datagen.fit(X_train)

# 모델 학습
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    validation_data=(X_test, y_test),
    epochs=20
)


부연 설명
평가 결과: 테스트 데이터에서 모델 성능을 확인합니다. accuracy가 80% 이상이면 성능이 적절하다고 판단할 수 있습니다.
저장: 학습된 모델은 model.save()로 저장하고 나중에 로드해 사용할 수 있습니다.

In [ ]:
# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

# 모델 저장
model.save('pupil_reflex_model.h5')


부연 설명
사용자 입력 이미지: 경로를 통해 입력된 이미지를 학습 데이터와 동일한 전처리를 거쳐 예측합니다.
결과 해석: 모델 출력이 0.5 이상이면 "정상 반응", 그렇지 않으면 "비정상 반응"으로 분류합니다.

In [ ]:
def predict_pupil_reflex(image_path):
    """
    저장된 모델을 사용해 새로운 이미지를 예측합니다.

    Parameters:
        image_path (str): 예측할 이미지 경로

    Returns:
        str: 예측 결과 (정상 반응 / 비정상 반응)
    """
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (64, 64)) / 255.0
    img = img.reshape(1, 64, 64, 1)
    prediction = model.predict(img)
    return "정상 반응" if prediction[0] > 0.5 else "비정상 반응"

# 예측 테스트
test_image = 'path_to_test_image.jpg'
result = predict_pupil_reflex(test_image)
print(f'테스트 이미지 결과: {result}')


사용자 및 정확도에 영향을 미치는 조정 요소
이미지 크기:
GPU 메모리와 학습 시간을 고려해 (64, 64) 크기를 조정합니다. 큰 크기는 더 많은 정보를 포함하지만 처리 시간이 증가합니다.
데이터 양:
데이터가 부족하면 증강 기법을 사용하거나 Transfer Learning을 통해 사전 학습된 모델(VGG, ResNet 등)을 활용할 수 있습니다.
과적합 방지:
Dropout 비율(0.25~0.5)과 학습 데이터를 균형 있게 설정하세요.
학습 파라미터:
batch_size, epochs, learning_rate 등을 조정해 최적의 성능을 찾으세요.
이 코드는 전체 파이프라인을 설계한 예입니다. 각 단계에서 테스트와 조정을 반복해 최적의 모델을 개발하세요.
